In [1]:
from cs103 import *
from typing import NamedTuple, List, Optional
from enum import Enum
import math  # for math.sqrt, the square root function.

## Module 6 Worksheet Sample Solution

Only including the big code problem here (problem 4).

### Problem 4

Refactor Riley's code below. But first, run all this code to make sure it's working!

We've made a second copy of the code (so you can keep the original around!) along with the functions we provided in the worksheet below.

In [2]:
### Data definitions

Palette = List[str]
# interp. a palette (collection) of colors. Each color in a palette
# is a recognized color name (from the so-called "X11 colors") or a
# "#" followed by 6 letters that must be a digit ("0" through "9")
# or one of the first 6 letters ("A" through "F" or "a" through "f").
# With the "#" colors, the 6 letters come in three pairs that indicate
# the amount of red (first pair), green (second pair), and blue (third
# pair) in a color.
P0 = []
P_BLACK = ["black", "#000000"]  # black in two different ways
P_UBC_V1 = ["blue", "gold"]
P_UBC_V2 = ["#0000FF", "#FFD700"]

# template based on arbitrary-sized
@typecheck
def fn_for_palette(p: Palette) -> ...:
    # description of accumulator
    acc = ...    # type: ...
    
    for color in p:
        acc = ...(color, acc)
    
    return ...(acc)


ColorPreference = Enum('ColorPreference', ['grayscale', 'color', 'any'])
# interp. a color preference for display, one of grayscale (black, white,
# or a shade of gray), color (NOT grayscale), or any (any color is 
# acceptable)
# examples are redundant for enumerations

# template based on enumeration (3 cases)
@typecheck
def fn_for_color_preference(cp: ColorPreference) -> ...:
    if cp == ColorPreference.grayscale:
        return ...
    elif cp == ColorPreference.color:
        return ...
    elif cp == ColorPreference.any:
        return ...

## Solution Process

The provided solution was, politely, a huge mess. It was hard to know where to start.

A really good approach might have been to hold the whole function in our heads and then determine what piece to "tease out" to form the high-level structure. We were having trouble doing that.

So, we scanned from top to bottom instead, looking for clearly separate tasks to peel off into helper functions and cleaning up bit by bit until a larger refactoring seemed visible. Each time we had cleanly peeled off a portion of the code, we made sure to re-run and see that all the tests were passing!

1. We saw the ColorPreference template being used to initialize an accumulator. So, we broke off a `get_default_bg_color` function to get a background color if the palette had none. 
2. The body of the loop was suddenly all about color preferences, which didn't make sense inside the Palette template. So, it seemed like something needed to be peeled off, but again it was hard to tell how. So, we focused on smaller things. In this case, figuring out if a color was grayscale seemed like a hefty problem of its own, largely independent of the overall problem being solved and requiring domain expertise about colors. We created `is_grayscale_color`. 
   This refactoring was exciting because we got a bug. One of our tests suddenly gave a background color of silver when it was supposed to be gray. (The test failed, and we could see the background was too white-ish.) It looked like it was returning the last rather than the first color. We went back and fixed up the code by referring back to the original code's use of `not found` conditions. Good thing we kept the original code around too!
   (While performing this refactoring, we noticed that the code in the ColorPreference.any case that checks `if len(p) > 0` is silly. If we're in the loop checking on colors in `p`, then `p` must have some elements in it. Otherwise, we would never have entered the body of the loop. So, we simplified that code.)
   By the end of this, our function body was "only" 26 lines long, down from about 60! Yay! It's not that more code is bad, but lots of code inside a single function is pretty bad.
3. Now it was clear that the body of the loop was checking if `color` was appropriate for `cp` and, if so and if no color had been found yet, recording it as the intended background color `bg_color`. So, we pulled out an `is_appropriate_color` helper.
4. Finally the main function was concise and clear enough for us to read off its implicit composition plan: (1) find an appropriate color, (2) find the radius of the background circle, and (3) overlay the image on the background circle. We refactored the function using the composition rule to finish off our design!

Our testing could perhaps be beefed up for some of these functions, but the tests are sufficient **given that** we know for sure that various factors are being tested in the helper functions (e.g., upper- vs lower-case isn't being checked in `choose_appropriate_color` but it is in helpers to that function).

Having said that, look **individually** at each of our functions. How many different tasks do they perform? How hard are they to understand? What if we changed the way we defined grayscale colors, how hard would it be to manage that?

In [3]:
import math  # for math.sqrt, the square root function.

@typecheck
def make_sticker(i: Image, cp: ColorPreference, p: Palette) -> Image:
    """
    returns the image i atop a round background that is (1) in the first
    color acceptable for the given color preference and (2) big enough 
    to hold the image (i.e., a box of the image's height and width). If
    the palette has no acceptable colors, chooses a default color 
    (white, if it matches cp, and red otherwise).
    """
    #return i  #stub
    # template from composition
    # Plan:
    # 1) choose an appropriate background color
    # 2) find the radius of the background circle needed
    # 3) return i overlaid on the background circle
    
    bg_color = choose_appropriate_color(p, cp)
    radius = get_circumscribed_radius(i)
    return overlay(i, circle(radius, "solid", bg_color))


@typecheck
def get_circumscribed_radius(i: Image) -> float:
    """
    returns the radius of the smallest circle that can fit around 
    (the "bounding box" of) i.
    
    (That's half of the square root of the height squared plus the width squared.)
    """
    #return 0.0  #stub
    h = image_height(i)
    w = image_width(i)
    return math.sqrt(h*h + w*w) / 2


@typecheck
def choose_appropriate_color(p: Palette, cp: ColorPreference) -> str:
    """
    returns the first appropriate color from p for cp or a default
    if none can be found (defaults to white if it matches cp and
    red otherwise)
    """
    #return ""  #stub
    # template from Palette with additional parameter cp
    # but largely copied from make_sticker during refactoring

    for color in p:
        if is_appropriate_color(color, cp):
            return color
    
    return get_default_bg_color(cp)


@typecheck
def is_appropriate_color(color: str, cp: ColorPreference) -> bool:
    """
    returns True if color is appropriate for cp.
    
    color must be a recognized color name (from the so-called 
    "X11 colors") or a "#" followed by 6 letters that must be
    a digit ("0" through "9") or one of the first 6 letters
    ("A" through "F" or "a" through "f").
    
    With the "#" colors, the 6 letters come in three pairs that
    indicate the amount of red (first pair), green (second pair), 
    and blue (third pair) in a color.
    """
    #return True  #stub
    # template from ColorPreference with additional parameter color
    # largely copied, however, from the body of make_sticker 
    # during refactoring.
    
    if cp == ColorPreference.grayscale:
        return is_grayscale_color(color)
    elif cp == ColorPreference.color:
        return not is_grayscale_color(color)
    elif cp == ColorPreference.any:
        return True    


@typecheck
def is_grayscale_color(color: str) -> bool:
    """
    returns True if the given color is grayscale.
    
    color must be a recognized color name (from the so-called 
    "X11 colors") or a "#" followed by 6 letters that must be
    a digit ("0" through "9") or one of the first 6 letters
    ("A" through "F" or "a" through "f").
    
    With the "#" colors, the 6 letters come in three pairs that
    indicate the amount of red (first pair), green (second pair), 
    and blue (third pair) in a color.
    """
    #return True   #stub
    #return ...(color)  #template

    # We might want to refactor to have a function that
    # determines whether a string is a color code. We'll
    # leave that as an exercise if you want to do it!
    if color[0] == "#":
        return is_grayscale_color_code(color)
    else:
        return is_grayscale_color_word(color)
    

@typecheck
def is_grayscale_color_code(color_code: str) -> bool:
    """
    returns True if the given color is grayscale.
    
    color must be "#" followed by 6 letters that must be
    a digit ("0" through "9") or one of the first 6 letters
    ("A" through "F" or "a" through "f").
    
    The 6 letters come in three pairs that indicate the
    amount of red (first pair), green (second pair), 
    and blue (third pair) in a color. All three pairs are the
    same for a grayscale color.
    """
    #return True   #stub
    #return ...(color_code)  #template

    # We don't care about lower- vs. upper-case.
    lc_color = color_code.lower()
    return lc_color[1:3] == lc_color[3:5] and \
           lc_color[3:5] == lc_color[5:7]


@typecheck
def is_grayscale_color_word(color: str) -> bool:
    """
    returns True if the given color contains any of the grayscale
    color words (gray, grey, black, white, silver, or gainsboro),
    ignoring case (upper vs lower)
    
    color must be a recognized color name (from the "X11 colors").
    Here are those color names: https://en.wikipedia.org/wiki/X11_color_names
    """
    #return True  #stub
    # template from Palette based on GRAYSCALE_COLOR_PALETTE below rather
    # than on parameter. Added parameter color.

    GRAYSCALE_COLOR_PALETTE = ["gray", "grey", "black", "white", "silver", "gainsboro"]

    lc_color = color.lower()
    
    for word in GRAYSCALE_COLOR_PALETTE:
        if word in lc_color:
            return True

    return False    


@typecheck
def get_default_bg_color(cp: ColorPreference) -> str:
    """
    return an appropriate default background color for the given
    preference cp (white if possible, red if not).
    """
    #return ""  #stub
    # template from ColorPreference

    if cp == ColorPreference.grayscale:
        return "white"
    elif cp == ColorPreference.color:
        return "red"
    elif cp == ColorPreference.any:
        return "white"



start_testing()

I1 = rectangle(40, 20, "solid", "blue")
I2 = rectangle(10, 80, "solid", "purple")


expect(make_sticker(I1, ColorPreference.grayscale, []), 
       overlay(I1, circle(math.sqrt(40*40 + 20*20) / 2, "solid", "white")))
expect(make_sticker(I1, ColorPreference.color, []), 
       overlay(I1, circle(math.sqrt(40*40 + 20*20) / 2, "solid", "red")))
expect(make_sticker(I1, ColorPreference.any, []), 
       overlay(I1, circle(math.sqrt(40*40 + 20*20) / 2, "solid", "white")))

expect(make_sticker(I2, ColorPreference.grayscale, []), 
       overlay(I2, circle(math.sqrt(10*10 + 80*80) / 2, "solid", "white")))
expect(make_sticker(I2, ColorPreference.color, []), 
       overlay(I2, circle(math.sqrt(10*10 + 80*80) / 2, "solid", "red")))
expect(make_sticker(I2, ColorPreference.any, []), 
       overlay(I2, circle(math.sqrt(10*10 + 80*80) / 2, "solid", "white")))

expect(make_sticker(I1, ColorPreference.grayscale, ["purple", "gray", "silver"]), 
       overlay(I1, circle(math.sqrt(40*40 + 20*20) / 2, "solid", "gray")))
expect(make_sticker(I1, ColorPreference.color, ["purple", "gray", "silver"]), 
       overlay(I1, circle(math.sqrt(40*40 + 20*20) / 2, "solid", "purple")))
expect(make_sticker(I1, ColorPreference.any, ["purple", "gray", "silver"]), 
       overlay(I1, circle(math.sqrt(40*40 + 20*20) / 2, "solid", "purple")))

expect(make_sticker(I1, ColorPreference.color, ["gray", "silver"]), 
       overlay(I1, circle(math.sqrt(40*40 + 20*20) / 2, "solid", "red")))
expect(make_sticker(I1, ColorPreference.grayscale, ["purple", "gold"]), 
       overlay(I1, circle(math.sqrt(40*40 + 20*20) / 2, "solid", "white")))

summary()


start_testing()

I1 = rectangle(40, 20, "solid", "blue")
I2 = rectangle(10, 80, "solid", "purple")

expect(get_circumscribed_radius(I1), math.sqrt(40*40 + 20*20) / 2)
expect(get_circumscribed_radius(I2), math.sqrt(10*10 + 80*80) / 2)

summary()


start_testing()

expect(choose_appropriate_color([], ColorPreference.grayscale), "white")
expect(choose_appropriate_color([], ColorPreference.color), "red")
expect(choose_appropriate_color([], ColorPreference.any), "white")

expect(choose_appropriate_color(["purple", "gold"], ColorPreference.grayscale), "white")
expect(choose_appropriate_color(["purple", "gold"], ColorPreference.color), "purple")
expect(choose_appropriate_color(["purple", "gold"], ColorPreference.any), "purple")

expect(choose_appropriate_color(["silver", "grey"], ColorPreference.grayscale), "silver")
expect(choose_appropriate_color(["silver", "grey"], ColorPreference.color), "red")
expect(choose_appropriate_color(["silver", "grey"], ColorPreference.any), "silver")

expect(choose_appropriate_color(["grey", "gold", "purple", "silver"], ColorPreference.grayscale), "grey")
expect(choose_appropriate_color(["grey", "gold", "purple", "silver"], ColorPreference.color), "gold")
expect(choose_appropriate_color(["grey", "gold", "purple", "silver"], ColorPreference.any), "grey")

# A bit of testing on color codes:
expect(choose_appropriate_color(["#888888", "#abcdef", "#eeee00", "#cccccc"], ColorPreference.grayscale), "#888888")
expect(choose_appropriate_color(["#888888", "#abcdef", "#eeee00", "#cccccc"], ColorPreference.color), "#abcdef")
expect(choose_appropriate_color(["#888888", "#abcdef", "#eeee00", "#cccccc"], ColorPreference.any), "#888888")

summary()


start_testing()

expect(is_appropriate_color("grey", ColorPreference.grayscale), True)
expect(is_appropriate_color("grey", ColorPreference.color), False)
expect(is_appropriate_color("grey", ColorPreference.any), True)

expect(is_appropriate_color("Purple", ColorPreference.grayscale), False)
expect(is_appropriate_color("Purple", ColorPreference.color), True)
expect(is_appropriate_color("Purple", ColorPreference.any), True)

expect(is_appropriate_color("#555555", ColorPreference.grayscale), True)
expect(is_appropriate_color("#555555", ColorPreference.color), False)
expect(is_appropriate_color("#555555", ColorPreference.any), True)

expect(is_appropriate_color("#eeee00", ColorPreference.grayscale), False)
expect(is_appropriate_color("#eeee00", ColorPreference.color), True)
expect(is_appropriate_color("#eeee00", ColorPreference.any), True)

summary()


start_testing()

# We clearly need more tests than this, but this is probably
# enough given that we've decided to break this into separate
# functions for color words and color "codes". 
expect(is_grayscale_color("purple"), False)
expect(is_grayscale_color("grey"), True)
expect(is_grayscale_color("silver"), True)
expect(is_grayscale_color("gold"), False)

expect(is_grayscale_color("#112233"), False)
expect(is_grayscale_color("#222222"), True)

summary()


start_testing()

# Basic tests:
expect(is_grayscale_color_code("#000000"), True)
expect(is_grayscale_color_code("#010000"), False)
expect(is_grayscale_color_code("#000100"), False)
expect(is_grayscale_color_code("#000101"), False)

# Case doesn't matter:
expect(is_grayscale_color_code("#f3f3F3"), True)

# One more with all different digits:
expect(is_grayscale_color_code("#abcdef"), False)

summary()


start_testing()

expect(is_grayscale_color_word("purple"), False)
expect(is_grayscale_color_word("grey"), True)
expect(is_grayscale_color_word("silver"), True)
expect(is_grayscale_color_word("gold"), False)

# Test each of the color words, inside larger
# "phrases" if possible
expect(is_grayscale_color_word("black"), True)
expect(is_grayscale_color_word("gainsboro"), True)
expect(is_grayscale_color_word("light gray"), True)
expect(is_grayscale_color_word("dark grey"), True)
expect(is_grayscale_color_word("white smoke"), True)

# Test case issues
expect(is_grayscale_color_word("Grey"), True)

summary()


start_testing()

expect(get_default_bg_color(ColorPreference.grayscale), "white")
expect(get_default_bg_color(ColorPreference.color), "red")
expect(get_default_bg_color(ColorPreference.any), "white")

summary()

11 of 11 tests passed
2 of 2 tests passed
15 of 15 tests passed
12 of 12 tests passed
6 of 6 tests passed
6 of 6 tests passed
10 of 10 tests passed
3 of 3 tests passed
